<a href="https://colab.research.google.com/github/Frankz199/Final-Year-Project/blob/main/FZ_num_10_FL_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/colab/Results

/content/drive/MyDrive/colab/Results


In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip install syft==0.2.6

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import logging
import copy
import csv


import syft as sy

hook = sy.TorchHook(torch)

In [ ]:
# Create clients in a dictionary so we can call upon them easier.
clients = dict()

for i in range(10):
  clients[i] = sy.VirtualWorker(hook,id=F"client_{i}")
  


In [ ]:
# define the args
args = {
    'use_cuda' : True,
    'batch_size' : 64,
    'test_batch_size' : 1000,
    'lr' : 0.01,
    'log_interval' : 100,
    'epochs' : 5
}

use_cuda = args['use_cuda'] and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels = 64, kernel_size = 3, stride = 1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=64*12*12, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = F.max_pool2d(x,2)
        x = x.view(-1, 64*12*12)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x


In [ ]:
# Federate the training dataset. The clients are obtained from the dictionary created earlier.
federated_train_loader = sy.FederatedDataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate(list(clients.values())),
    batch_size=args['batch_size'], shuffle=True)

# test data remains with us locally
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args['test_batch_size'], shuffle=True)

0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()

    accuracy_history = []
    acc_average = []

    loss_history = []
    loss_average = []

    average_weight_history = []
    moving_average_of_weight = []

    individual_loss = []
    individual_acc = []
    individual_weight = []
    individual_total_bytes = []
    
    success = 0
    failure = 0
    x = 0

    with open('Francescocode-num-ten.csv', 'a', newline='') as csvfile:

      fieldnames = ['individual_loss', 'individual_acc', 'individual_weight', 'x', 'data.location.id', 'success', 'failure', 'individual_total_bytes']

      thewriter = csv.DictWriter(csvfile, fieldnames=fieldnames)

      thewriter.writeheader()


      # iterate over federated data
      for batch_idx, (data, target) in enumerate(train_loader):
          x += 1
          does_it_pass_the_decision_tree = FrancescoCode(model,data,accuracy_history,acc_average,loss_history,loss_average,average_weight_history,moving_average_of_weight,x,individual_weight,individual_acc,individual_loss,individual_total_bytes)

          if does_it_pass_the_decision_tree:      
            # send the model to the remote location 
            latest_global_model = copy.deepcopy(model.state_dict())
            model = model.send(data.location)

      
          # the same torch code that we are use to
            data, target = data.to(device), target.to(device)
          
            optimizer.zero_grad()
          
            output = model(data)
    

            loss = F.nll_loss(output, target)
          
      

            loss.backward()
            optimizer.step()
          else:
            print('Reject')

          # get back the updated model
          if does_it_pass_the_decision_tree:
            model.get()
            success += 1
            #print(success)
          else:
            model.load_state_dict(latest_global_model)
            failure += 1
            #print(failure)
          

          if batch_idx % args['log_interval'] == 0:


              loss = loss.get()

              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, 
                    batch_idx * args['batch_size'], # no of images done
                    len(train_loader) * args['batch_size'], # total images left
                    100. * batch_idx / len(train_loader), 
                    loss.item()
                  )
              )
          
          thewriter.writerow({'individual_loss': individual_loss, 'individual_acc': individual_acc, 'individual_weight': individual_weight, 'x': x, 'data.location.id': data.location.id, 'success': success, 'failure': failure, 'individual_total_bytes': individual_total_bytes})

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # add losses together
            test_loss += F.nll_loss(output, target, reduction='sum').item() 

            # get the index of the max probability class
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    return (100. * correct / len(test_loader.dataset), test_loss)

In [ ]:
def FrancescoCode(model,data,accuracy_history,acc_average,loss_history,loss_average,average_weight_history,moving_average_of_weight,x,individual_weight,individual_acc,individual_loss,individual_total_bytes):
  Pass = 0
  Fail = 0
  #Get the client ID of the Model that is being Sent
  print(F"Model ID: {data.location.id}")

  latest_model = []
  individual_weight.clear()
  individual_acc.clear()
  individual_loss.clear()
  individual_total_bytes.clear()
  window_size = 3

  # # #Iterate over the model layers to obtain the weights and biases
  # # # Here compare the weights of the previous model and the current model
  for layer in model.parameters():
    latest_model.append(layer.detach().clone())
    #print(layer)
  
  copy_model = copy.deepcopy(latest_model)
  l0 = torch.flatten(copy_model[0])
  l1 = torch.flatten(copy_model[1])
  l2 = torch.flatten(copy_model[2])
  l3 = torch.flatten(copy_model[3])
  l4 = torch.flatten(copy_model[4])
  l5 = torch.flatten(copy_model[5])
  l6 = torch.flatten(copy_model[6])
  l7 = torch.flatten(copy_model[7])

  merge_them = torch.cat((l0,l1,l2,l3,l4,l5,l6,l7), 0)
  list_data = torch.flatten(merge_them)

  total_bytes = (list_data.element_size() * list_data.nelement())
  individual_total_bytes.append(total_bytes)

  flatten_weight = torch.flatten(latest_model[6])
  average_weight = torch.mean(flatten_weight)
  individual_weight.append(average_weight)
  intial_average_weight = copy.deepcopy(average_weight)
  moving_average_of_weight.append(intial_average_weight)
  latest_model.clear()

  if average_weight >= moving_average_of_weight[0]:
    average_weight_history.append(average_weight)
    averageW = torch.mean(torch.stack(average_weight_history))
    moving_average_of_weight.clear()
    moving_average_of_weight.append(averageW)
    Pass += 1
  else:
    average_weight_history.append(average_weight)
    averageW = torch.mean(torch.stack(average_weight_history))
    moving_average_of_weight.clear()
    moving_average_of_weight.append(averageW)    
    Fail += 1

  if x >= 4:
    moving_average_of_weight.clear()
    windoW = average_weight_history[(x-4) : (x-4) + window_size]
    rolling_averageW = torch.mean(torch.stack(windoW))
    moving_average_of_weight.append(rolling_averageW)
    #print(windoW)


  # Get accuracy of the model before training ( Might want to reduce the size so its faster )
  accuracy, loss = test(model,device,test_loader)
  intial_acc = copy.deepcopy(accuracy)
  intial_loss = copy.deepcopy(loss)
  individual_acc.append(accuracy)
  individual_loss.append(loss)

 
  acc_average.append(intial_acc) 
  
  if x < 4:
    if accuracy >= acc_average[0]:
      accuracy_history.append(accuracy)
      averageA = round(sum(accuracy_history) / len(accuracy_history), 4)
      acc_average.clear()
      acc_average.append(averageA)
      Pass += 1
    else:
      accuracy_history.append(accuracy)
      averageA = round(sum(accuracy_history) / len(accuracy_history), 4)
      acc_average.clear()
      acc_average.append(averageA)
      Fail += 1

  if x >= 4:
    if accuracy >= acc_average[0]:
      accuracy_history.append(accuracy)
      acc_average.clear()
      windowA = accuracy_history[(x-4) : (x-4) + window_size]
      averageA = round(sum(windowA) / window_size, 4)
      acc_average.append(averageA)
      #print(windowA)
      Pass += 2
    else:
      accuracy_history.append(accuracy)
      acc_average.clear()
      windowA = accuracy_history[(x-4) : (x-4) + window_size]
      averageA = round(sum(windowA) / window_size, 4)
      acc_average.append(averageA)
      #print(windowA)
      Fail += 2

  loss_average.append(intial_loss)
  if x < 4:
    if loss <= loss_average[0]:
      loss_history.append(loss)
      averageL = round(sum(loss_history)/ len(loss_history), 15)
      loss_average.clear()
      loss_average.append(averageL)
      Pass += 1
    else:
      loss_history.append(loss)
      averageL = round(sum(loss_history)/ len(loss_history), 15)
      loss_average.clear()
      loss_average.append(averageL)
      Fail += 1

  if x >= 4:
    if loss <= loss_average[0]:
      loss_history.append(loss)
      loss_average.clear()
      windowL = loss_history[(x-4) : (x-4) + window_size]
      averageL = round(sum(windowL) / window_size, 15)
      loss_average.append(averageL)
      #print(windowL)
      Pass += 2
    else:
      loss_history.append(loss)
      loss_average.clear()
      windowL = loss_history[(x-4) : (x-4) + window_size]
      averageL = round(sum(windowL) / window_size, 15)
      loss_average.append(averageL)
      #print(windowL)
      Fail += 2

  if Pass >= Fail:
    return True
  else:
    return False


In [ ]:
model = Net().to(device)
#previous_global_model = Net().to(device)

optimizer = optim.SGD(model.parameters(), lr=args['lr'])

logging.info("Starting training !!")

for epoch in range(1, args['epochs'] + 1):
        train(args, model, device, federated_train_loader, optimizer, epoch)
        test(model, device, test_loader)
  

Streaming output truncated to the last 5000 lines.
Reject
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Model ID: client_1
Model ID: client_1
Model ID: client_1
Model ID: client_1
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Reject
Model ID: client_1
Model ID: client_1
Model ID: client_1
Model ID